In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense, BatchNormalization, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU
import pandas as pd

In [ ]:
real_data = pd.read_csv("output.csv")

In [ ]:
real_data. drop(['n','MRI'],axis=1,inplace=True)

In [ ]:
def build_generator():
    model = tf.keras.Sequential()

    # Input layer (conditioning variables and noise)
    model.add(Dense(units=64, input_shape=(noise_dim,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Additional fully connected layers with BatchNormalization and LeakyReLU activation
    model.add(Dense(units=64))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Output layer with 6 units (matches the desired output shape)
    model.add(Dense(units=6, activation='sigmoid'))

    return model

# Discriminator network
def build_discriminator():
    model = tf.keras.Sequential()

    # Input layer (real or generated data)
    model.add(Dense(units=64, input_shape=(6,)))  # Input shape matches the data shape
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Additional fully connected layers with BatchNormalization and LeakyReLU activation
    model.add(Dense(units=64))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    # Output layer (binary classification)
    model.add(Dense(units=1, activation='sigmoid'))

    return model


In [ ]:
binary_crossentropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

In [ ]:

def discriminator_loss(real_output, fake_output):
    real_loss = binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Generator loss function
def generator_loss(fake_output):
    return binary_crossentropy(tf.ones_like(fake_output), fake_output)

In [ ]:
def train_gan(generator, discriminator, real_data, num_steps, batch_size, noise_dim, print_interval):
    generated_data_history = []

    for step in range(num_steps):
        noise = tf.random.normal([batch_size, noise_dim])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_data = generator(noise)
            real_output = discriminator(real_data)
            fake_output = discriminator(generated_data)
            gen_loss = generator_loss(fake_output)
            disc_loss = discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        generated_data_history.append(generated_data.numpy())  # Convert to numpy and append

        if (step + 1) % print_interval == 0:
            print(f"Step {step + 1}: Generator Loss: {gen_loss}, Discriminator Loss: {disc_loss}")

    # Reshape the final generated data to (4806, 6)
    generated_data_history = np.concatenate(generated_data_history, axis=0)
    generated_data_history = generated_data_history[:num_steps, :]  # Ensure the correct shape

    return generated_data_history


In [ ]:
generator = build_generator()
discriminator = build_discriminator()

num_steps = 4806  # Number of training steps (adjust as needed)
batch_size = 6  # Batch size for each training step (adjust as needed)
noise_dim = 6  # Dimension of random noise input to the generator (adjust as needed)
print_interval = 100  # Print loss values every 'print_interval' steps (adjust as needed)

# Assuming your real_data is a pandas DataFrame, you can convert it to a NumPy array like this:
real_data_np = real_data.values

# Now, call the train_gan function with the NumPy array
generated_data_history = train_gan(generator, discriminator, real_data_np, num_steps, batch_size, noise_dim, print_interval)

Step 100: Generator Loss: 0.8822280764579773, Discriminator Loss: 0.5343238711357117
Step 200: Generator Loss: 1.1084975004196167, Discriminator Loss: 0.4006047248840332
Step 300: Generator Loss: 1.3687912225723267, Discriminator Loss: 0.29358628392219543
Step 400: Generator Loss: 1.5969997644424438, Discriminator Loss: 0.22632302343845367
Step 500: Generator Loss: 1.8214393854141235, Discriminator Loss: 0.17649133503437042
Step 600: Generator Loss: 2.0607283115386963, Discriminator Loss: 0.13623358309268951
Step 700: Generator Loss: 2.276641607284546, Discriminator Loss: 0.10828515142202377
Step 800: Generator Loss: 2.4781246185302734, Discriminator Loss: 0.08763021230697632
Step 900: Generator Loss: 2.662384033203125, Discriminator Loss: 0.07233595103025436
Step 1000: Generator Loss: 2.8351802825927734, Discriminator Loss: 0.060501858592033386
Step 1100: Generator Loss: 2.8819000720977783, Discriminator Loss: 0.05765913426876068
Step 1200: Generator Loss: 3.039487600326538, Discrimin

In [ ]:
#generated_data_history = np.array(generated_data_history)

In [ ]:
print(real_data.shape)

(4806, 6)


In [ ]:
generated_data_history.shape

(4806, 6)

In [ ]:
# generated_data_history = np.concatenate(generated_data_history, axis=0)
# generated_data_history = generated_data_history.reshape((num_steps, 6))

In [ ]:
print(generated_data_history)

[[6.1409974e-01 4.8928002e-01 4.4327116e-01 4.6422887e-01 5.0104189e-01
  4.3839070e-01]
 [5.1705325e-01 4.8766190e-01 4.1717219e-01 5.0290108e-01 5.6601638e-01
  4.7729322e-01]
 [5.5335402e-01 5.1085925e-01 4.1242447e-01 4.1745034e-01 4.7230238e-01
  4.8911899e-01]
 ...
 [9.9912697e-01 5.9518492e-04 9.9387497e-01 1.2914952e-03 9.9601597e-01
  5.1720394e-04]
 [9.9907255e-01 6.8194408e-04 9.9605620e-01 4.8407535e-03 9.9478650e-01
  4.5299222e-04]
 [9.9969649e-01 2.4617769e-04 9.9774677e-01 8.8149653e-04 9.9727422e-01
  8.8669294e-05]]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def calculate_performance_metrics(real_data_np, generated_data_history):
    rmse = np.sqrt(mean_squared_error(real_data_np, generated_data_history))
    correlation = np.corrcoef(real_data_np, generated_data_history)[0, 1]
    mae = mean_absolute_error(real_data_np, generated_data_history)
    return rmse, correlation, mae


In [ ]:
rmse, correlation, mae = calculate_performance_metrics(real_data_np, generated_data_history)

# Print the computed metrics
print(f"RMSE: {rmse}")
print(f"Correlation: {correlation}")
print(f"MAE: {mae}")

RMSE: 4529.259276470093
Correlation: 0.9936722988189369
MAE: 1739.1569243684762
